# Step 12 Character Voiceover Video

In [ ]:
%pip install ffmpeg-python
!ffmpeg -version


In [ ]:
from model import Story
import settings

story = Story.load_from_directory(settings.STORY_DIR + "/step_4")

In [ ]:
import ffmpeg
import math
import os

def create_looped_video_with_voiceover(video_path, audio_path, output_path):
    # Get video and audio durations
    video_info = ffmpeg.probe(video_path)
    audio_info = ffmpeg.probe(audio_path)
    video_duration = float(video_info['streams'][0]['duration'])
    audio_duration = float(audio_info['streams'][0]['duration'])
    
    # Calculate the number of loops required and create looped segments
    loops_needed = math.ceil(audio_duration / video_duration)
    segments = []
    for i in range(loops_needed):
        # Apply reverse filter on every alternate segment
        segment = ffmpeg.input(video_path)
        if i % 2 == 1:
            segment = segment.filter('reverse')
        segments.append(segment)

    # Concatenate all segments to reach or exceed the audio duration
    video = ffmpeg.concat(*segments, v=1, a=0)
    video = video.trim(end=audio_duration)  # Trim excess duration

    # Add the audio and export the final video
    final_output = ffmpeg.output(video, ffmpeg.input(audio_path), output_path, vcodec='libx264', acodec='aac')
    final_output.run(overwrite_output=True)


In [ ]:
import os
from IPython.display import display, Markdown, Video

voiceover_dir = f"{settings.STORY_DIR}/step_12/voiceover"
os.makedirs(voiceover_dir, exist_ok=True)
character_videos = []

for character in story.characters:
    src_video_path = f"{settings.STORY_DIR}/step_7/characters/{character.nickname}.mp4"
    # srce_audio_path = f"{settings.STORY_DIR}/step_11/voices/{character.nickname}_catchphrase.wav"
    srce_audio_path = f"{settings.STORY_DIR}/step_11/voices/{character.nickname}_catchphrase.fish.wav"
    dst_video_path = f"{voiceover_dir}/{character.nickname}.mp4"

    create_looped_video_with_voiceover(src_video_path, srce_audio_path, dst_video_path)

    character_videos.append(dst_video_path)

    # Display character name and embedded video
    display(Markdown(f"### {character.name}"))
    display(Video(dst_video_path, embed=True))


In [ ]:
# def combine_videos(video_paths, output_path):
#     """
#     Combines multiple video files into one long video.

#     Parameters:
#     - video_paths: List of paths to video files to combine.
#     - output_path: Path to save the combined output video.
#     """
#     # Ensure there is at least one video to combine
#     if not video_paths:
#         raise ValueError("The video_paths list is empty.")
    
#     # Create an input stream for each video and add to the inputs list
#     inputs = [ffmpeg.input(video) for video in video_paths]
    
#     # Concatenate videos in sequence
#     combined_video = ffmpeg.concat(*inputs, v=1, a=1)  # v=1 and a=1 mean we want to keep both video and audio

#     # Output the final concatenated video
#     final_output = ffmpeg.output(combined_video, output_path, vcodec='libx264', acodec='aac')
#     final_output.run(overwrite_output=True)

In [ ]:
import ffmpeg
import os

def combine_videos(video_paths, output_path):
    """
    Combines multiple video files into one long video, ensuring each has an audio track.

    Parameters:
    - video_paths: List of paths to video files to combine.
    - output_path: Path to save the combined output video.
    """
    
    def add_silent_audio(video_path):
        # Create a temporary file with a silent audio track
        temp_output = f"{video_path}_with_audio.mp4"
        ffmpeg.input(video_path).output(
            temp_output, vcodec='copy', acodec='aac', audio_bitrate='128k', anullsrc='1'
        ).run(overwrite_output=True)
        return temp_output
    
    # Ensure there is at least one video to combine
    if not video_paths:
        raise ValueError("The video_paths list is empty.")
    
    # Process each video to ensure it has an audio track
    processed_videos = []
    for video in video_paths:
        probe = ffmpeg.probe(video)
        has_audio = any(stream['codec_type'] == 'audio' for stream in probe['streams'])
        if not has_audio:
            # Add silent audio if the video has no audio track
            video_with_audio = add_silent_audio(video)
            processed_videos.append(video_with_audio)
        else:
            processed_videos.append(video)
    
    # Create an input stream for each processed video
    inputs = [ffmpeg.input(video) for video in processed_videos]
    
    # Concatenate videos in sequence
    combined_video = ffmpeg.concat(*[i.video for i in inputs], v=1, a=0)
    audio_concat = ffmpeg.concat(*[i.audio for i in inputs], v=0, a=1)
    
    # Output the final concatenated video
    final_output = ffmpeg.output(combined_video, audio_concat, output_path, vcodec='libx264', acodec='aac')
    final_output.run(overwrite_output=True)
    
    # Clean up any temporary files created
    for temp_file in processed_videos:
        if temp_file.endswith("_with_audio.mp4") and os.path.exists(temp_file):
            os.remove(temp_file)


In [ ]:
voiceover_dir = f"{settings.STORY_DIR}/step_12/voiceover"
output_path = f"{settings.STORY_DIR}/step_12/combined_video.mp4"
combine_videos(character_videos, output_path)

display(Video(output_path))
display(Video(output_path, embed=True))